# Textonic API
Če imaš težave z računanjem vektorjev; Funkcije za vektorizacijo z Textonic API. Spet kličem vektorizacijo na stolpcu 'Text' in dobim stolpec z vektorjem 'Embedding'

In [1]:
!pip install pandas numpy networkx numpy scikit-learn

In [2]:
import os
import pandas as pd

excel_path = os.path.join('..', 'data', 'Reworkan data.xlsx')  # Za primer sem dal krajši file
data = pd.read_excel(excel_path)
    
data.head()

,Article ID,Datum,Dan,Teden,Article created,Tip medija,Podtip medija,Media uuid,Medij,Rubric uuid,...,Pregled objav - citiranost virov,Razredi Indeksa medijske podobe,Doseg,Naklonjenost teme,Osrednja tema,Ključna sporočila,Fotografija,Omemba v naslovu,Omemba v podnaslovu,Tekst
0,0c801540-d013-11ed-9f09-af460eec62b1,2023-04-01,Saturday,13,2023-04-01 00:26:15.103,tisk,tisk,7eadd15d-28aa-485c-8c00-1e11cff87069,Delo - Sobotna priloga,97dd7328-45de-43f9-8732-d0b8f105f27c,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),zelo dober,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,Jerneja Terčon Strokovnjakinja za nevrorazličn...
1,2945adc3-d01b-11ed-83eb-0bd28cf4c8f0,2023-04-01,Saturday,13,2023-04-01 01:24:19.348,tisk,tisk,7eadd15d-28aa-485c-8c00-1e11cff87069,Delo - Sobotna priloga,97dd7328-45de-43f9-8732-d0b8f105f27c,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),zelo dober,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"Ni več upanja, povozil gaje tank / SAMO ZVER m..."
2,3e1f0d89-d010-11ed-83eb-0bd28cf4c8f0,2023-04-01,Saturday,13,2023-04-01 00:06:09.863,tisk,tisk,cf08eb8c-1e50-4884-ae0a-9ccf7d735817,Slovenske novice,e2d7d641-5fce-428c-b204-1c4b8f5a8eaa,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),nizek,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,Z rutinske operaciji e je vrnil na vozičku Grb...
3,43edd0d8-d00b-11ed-9d48-dfb487f9037c,2023-04-01,Saturday,13,2023-03-31 23:30:32.123,tisk,tisk,8d0af10b-3831-4342-9add-b2232332ae02,Večer - V soboto,c3f5a2d2-e46f-4888-8f04-afa6427e1818,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),dober,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"ZDRAVJE VRTILJAK IDEJ, ANALIZ IN INTERESOV ZDR..."
4,8cd51594-d01c-11ed-9f09-af460eec62b1,2023-04-01,Saturday,13,2023-04-01 01:34:15.879,tisk,tisk,2fd717ed-78ba-4f63-b257-cd096acb6bda,Delo,34846d6e-c759-4896-b1c8-4c85f69762d7,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),odličen,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,Protest Od jutri do preklica zdravniška stavka...


V realnosti pazi na okoljske spremenljivke

In [3]:
print(os.getenv('TTNX_API_KEY'))

fab30336-fea1-42ca-88bc-f93ec26b1005


Definiram Textonic API funkcije za vektorizacijo:

In [4]:
import uuid
import numpy as np
from ttnx.api import call_textonic


def ttnx_embed(df: pd.DataFrame, col_name: str, target_col_name: str = 'Embedding'):
    # Split the DataFrame into batches of 500 rows
    batch_size = 500
    num_batches = (len(df) + batch_size - 1) // batch_size
    id_column = 'Article ID'
    
    selected_columns = [id_column, col_name]
    responses = pd.DataFrame()
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        
        batch = df.iloc[start_idx:end_idx]
        request = {
            'requestId': str(uuid.uuid4()),
            'process': {
                'analysis': {
                    'steps': [
                        {
                            'step': 'doc_embed',
                            'attributes': [
                                {'named_sentence_filters': 'kl_transcript'}
                            ]
                        }
                    ]
                }
            },
            'documents': []
        }
        
        already_added_ids = set() 
        # Iterate over DataFrame rows and use selected columns and neglect duplicates
        for index, row in batch.iterrows():
            a = row[selected_columns].to_dict()
            if id_column not in a or not a[id_column]:
                continue
            if a[id_column] in already_added_ids:
                continue
            already_added_ids.add(a[id_column])
            if col_name not in a or not a[col_name]:
                continue
            document = {
                'id': a['Article ID'],
                'title': a['Article ID'],
                'sections': [
                    {
                        'outline': 'body',
                        'data': a[col_name]
                    }
                ]
            }
            request['documents'].append(document)
        
        resp_obj = call_textonic('/api/public/ml/process', request)
        
        result = {
            id_column: [],
            'Embedding': []
        }
        
        for res_item in resp_obj['data']:
            for res in res_item['result']:
                if 'c' in res and 'v' in res and 'doc_embed' in res['c']:
                    vect = np.array(res['v'])
                    result[id_column].append(res_item['id'])
                    result[target_col_name].append(vect)
                    
        print(f'Requested/received/valid vector : {len(batch)}/{len(resp_obj["data"])}/{len(result[target_col_name])}')
        # Convert the collected responses to a DataFrame
        response_df = pd.DataFrame(result)
        responses = pd.concat([responses, response_df])
    
    orig_len = len(df)
    df = pd.merge(df, responses, on=id_column, how='left').dropna(subset=[target_col_name])
    print(f'Requested/Successful {orig_len}/{len(df)}')
    return df

Remote call to compute Textonic embeddings za 500 člankov na enkrat:

In [5]:
data = ttnx_embed(data, 'Tekst', 'Embedding')
data[['Article ID', 'Datum', 'Medij', 'Tekst', 'Embedding']].head(200)

Requested/received/valid vector : 99/92/92
Requested/Successful 99/99


,Article ID,Datum,Medij,Tekst,Embedding
0,0c801540-d013-11ed-9f09-af460eec62b1,2023-04-01 00:00:00,Delo - Sobotna priloga,Jerneja Terčon Strokovnjakinja za nevrorazličn...,"[0.004464829806238413, 0.026770807802677155, 0..."
1,2945adc3-d01b-11ed-83eb-0bd28cf4c8f0,2023-04-01 00:00:00,Delo - Sobotna priloga,"Ni več upanja, povozil gaje tank / SAMO ZVER m...","[0.014561030082404613, 0.04490218684077263, 0...."
2,3e1f0d89-d010-11ed-83eb-0bd28cf4c8f0,2023-04-01 00:00:00,Slovenske novice,Z rutinske operaciji e je vrnil na vozičku Grb...,"[0.05338142812252045, 0.06102554127573967, 0.0..."
3,43edd0d8-d00b-11ed-9d48-dfb487f9037c,2023-04-01 00:00:00,Večer - V soboto,"ZDRAVJE VRTILJAK IDEJ, ANALIZ IN INTERESOV ZDR...","[0.029501112177968025, 0.05879836156964302, -0..."
4,8cd51594-d01c-11ed-9f09-af460eec62b1,2023-04-01 00:00:00,Delo,Protest Od jutri do preklica zdravniška stavka...,"[0.00616146856918931, 0.08077462017536163, -0...."
...,...,...,...,...,...
94,77e71000-d4f9-11ed-83eb-0bd28cf4c8f0,2023-04-07 06:03:00,Reporter.si,Slovenija Silvester Šurla 07. apr. 2023 Deli n...,"[0.04160451516509056, 0.07823976129293442, -0...."
95,c1b38c44-d4fc-11ed-9d48-dfb487f9037c,2023-04-07 06:26:00,24ur.com,Električni skiroji lahko pripomorejo k večji m...,"[0.00403248006477952, 0.04015741124749184, 0.0..."
96,b75ccef4-d50c-11ed-9246-2b5ebef623ad,2023-04-07 08:22:00,Sta.si,V Univerzitetnem kliničnem centru (UKC) Ljublj...,"[0.009282686747610569, 0.04146268218755722, 0...."
97,6f0e7f09-d50f-11ed-9f09-af460eec62b1,2023-04-07 08:40:00,24ur.com,V Univerzitetnem kliničnem centru Ljubljana so...,"[0.013681909069418907, 0.0399809293448925, 0.0..."


Funkcija za clusterizacijo

In [6]:
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

def cluster_louvain(df: pd.DataFrame, col_name: str = 'Embedding', similarity_threshold: float = 0.96):
    print("Clustering ...")
    embeddings = np.array(df[col_name].to_list())
    labels = [0] * len(embeddings)
    x = cosine_similarity(embeddings, embeddings)
    similarity_matrix = x > similarity_threshold
    graph = nx.from_numpy_array(similarity_matrix)
    communities = nx.algorithms.community.louvain_communities(graph, resolution=0.1)
    for community in communities:
        initial_member = min(community)
        for member in community:
            labels[member] = initial_member

    df['Cluster'] = pd.Series(labels)
    print("Clustered")
    return df

Zračunam clustering

In [7]:
data = cluster_louvain(data, 'Embedding', 0.96)  # <- similarity_threshold
data = data.drop(columns=['Embedding'])  # remove the vectors from data we don't need them anymore
data[['Article ID', 'Datum', 'Medij', 'Cluster', 'Tekst']].head(100)

Clustering ...
Clustered


,Article ID,Datum,Medij,Cluster,Tekst
0,0c801540-d013-11ed-9f09-af460eec62b1,2023-04-01 00:00:00,Delo - Sobotna priloga,0,Jerneja Terčon Strokovnjakinja za nevrorazličn...
1,2945adc3-d01b-11ed-83eb-0bd28cf4c8f0,2023-04-01 00:00:00,Delo - Sobotna priloga,1,"Ni več upanja, povozil gaje tank / SAMO ZVER m..."
2,3e1f0d89-d010-11ed-83eb-0bd28cf4c8f0,2023-04-01 00:00:00,Slovenske novice,2,Z rutinske operaciji e je vrnil na vozičku Grb...
3,43edd0d8-d00b-11ed-9d48-dfb487f9037c,2023-04-01 00:00:00,Večer - V soboto,3,"ZDRAVJE VRTILJAK IDEJ, ANALIZ IN INTERESOV ZDR..."
4,8cd51594-d01c-11ed-9f09-af460eec62b1,2023-04-01 00:00:00,Delo,4,Protest Od jutri do preklica zdravniška stavka...
...,...,...,...,...,...
94,77e71000-d4f9-11ed-83eb-0bd28cf4c8f0,2023-04-07 06:03:00,Reporter.si,25,Slovenija Silvester Šurla 07. apr. 2023 Deli n...
95,c1b38c44-d4fc-11ed-9d48-dfb487f9037c,2023-04-07 06:26:00,24ur.com,95,Električni skiroji lahko pripomorejo k večji m...
96,b75ccef4-d50c-11ed-9246-2b5ebef623ad,2023-04-07 08:22:00,Sta.si,96,V Univerzitetnem kliničnem centru (UKC) Ljublj...
97,6f0e7f09-d50f-11ed-9f09-af460eec62b1,2023-04-07 08:40:00,24ur.com,96,V Univerzitetnem kliničnem centru Ljubljana so...
